In [2]:
%run cm_queryset_google_topics.py

In [ ]:
# Basics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
# sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
# Views 3
from viewser.operations import fetch
from viewser import Queryset, Column
import views_runs
from views_partitioning import data_partitioner, legacy
from stepshift import views

In [ ]:
qs_baseline = (Queryset("fatalities_mc_baseline", "country_month")

                   # target variable
                   .with_column(Column("ln_ged_sb_dep", from_table="ged2_cm", from_column="ged_sb_best_sum_nokgi")
                                .transform.ops.ln()
                                .transform.missing.fill()
                                )

                   # timelag 0 of target variable
                   .with_column(Column("ln_ged_sb", from_table="ged2_cm", from_column="ged_sb_best_sum_nokgi")
                                .transform.ops.ln()
                                .transform.missing.fill()
                                )
                   # Decay functions
                   # sb
                   .with_column(Column("decay_ged_sb_5", from_table="ged2_cm", from_column="ged_sb_best_sum_nokgi")
                                .transform.missing.replace_na()
                                .transform.bool.gte(5)
                                .transform.temporal.time_since()
                                .transform.temporal.decay(24)
                                .transform.missing.replace_na()
                                )
                   # os
                   .with_column(Column("decay_ged_os_5", from_table="ged2_cm", from_column="ged_os_best_sum_nokgi")
                                .transform.missing.replace_na()
                                .transform.bool.gte(5)
                                .transform.temporal.time_since()
                                .transform.temporal.decay(24)
                                .transform.missing.replace_na()
                                )
                   # Spatial lag decay
                   .with_column(Column("splag_1_decay_ged_sb_5", from_table="ged2_cm",
                                       from_column="ged_sb_best_sum_nokgi")
                                .transform.missing.replace_na()
                                .transform.bool.gte(5)
                                .transform.temporal.time_since()
                                .transform.temporal.decay(24)
                                .transform.spatial.countrylag(1, 1, 0, 0)
                                .transform.missing.replace_na()
                                )

                   # From 
                   
                   .with_column(Column("wdi_sp_pop_totl", from_table="wdi_cy", from_column="wdi_sp_pop_totl")
                                .transform.missing.fill()
                                .transform.temporal.tlag(12)
                                .transform.missing.fill()
                                )

                   .with_theme("fatalities")
                   .describe("""Fatalities conflict history, cm level
    
                   Predicting ln(fatalities) using conflict predictors, ultrashort
    
                             """)
                   )

    data = qs_baseline.publish().fetch()

    print(f"fat_dev_mc_media_baseline; "
          f"A dataset with {len(data.columns)} columns, with "
          f"data between t {min(data.index.get_level_values(0))} "
          f"and {max(data.index.get_level_values(0))}. "
          f"({len(np.unique(data.index.get_level_values(1)))} units)"
          )

    
    ###################################################################################################################
    # Mueller & Rauh topic model features
    # this is a version used for MC presentation

    qs_topics_stub = (Queryset("fatalities_mc_topics_stub", "country_month")

                      # Topic 0, religious tensions: original, tlag1, 12 month moving average on tlag1
                      .with_column(Column('topic0_religion_t3', from_table='topic_cm', from_column='topic_religion')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(3)
                                   .transform.missing.fill()
                                   )

                      .with_column(Column('topic0_religion_t15', from_table='topic_cm', from_column='topic_religion')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(15)
                                   .transform.missing.fill()
                                   )

                      .with_column(Column('topic0_religion_t3_stock', from_table='topic_cm',
                                          from_column='topic_religion')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(3)
                                   .transform.missing.fill()
                                   .transform.temporal.moving_average(12)
                                   .transform.missing.fill()
                                   )
                      .with_column(Column('splag_topic1_religion_t3_stock', from_table='topic_cm',
                                          from_column='topic_religion')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(3)
                                   .transform.missing.fill()
                                   .transform.temporal.moving_average(12)
                                   .transform.missing.fill()
                                   .transform.spatial.countrylag(1, 1, 0, 0)
                                   .transform.missing.replace_na()
                                   )

                      # Topic 1, politics, original: tlag1, 12 month moving average on tlag1
                      .with_column(Column('topic1_politics_t3', from_table='topic_cm', from_column='topic_politics')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(3)
                                   .transform.missing.fill()
                                   )

                      .with_column(Column('topic1_politics_t15', from_table='topic_cm', from_column='topic_politics')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(15)
                                   .transform.missing.fill()
                                   )

                      .with_column(Column('topic1_politics_t3_stock', from_table='topic_cm',
                                          from_column='topic_politics')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(3)
                                   .transform.missing.fill()
                                   .transform.temporal.moving_average(12)
                                   .transform.missing.fill()
                                   )

                      .with_column(Column('splag_topic2_politics_t3_stock', from_table='topic_cm',
                                          from_column='topic_politics')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(3)
                                   .transform.missing.fill()
                                   .transform.temporal.moving_average(12)
                                   .transform.missing.fill()
                                   .transform.spatial.countrylag(1, 1, 0, 0)
                                   .transform.missing.replace_na()
                                   )

                      # Topic 2, diplomacy and sanctions: original, tlag1, 12 month moving average on tlag1
                      .with_column(Column('topic2_sanctions_3', from_table='topic_cm', from_column='topic_sanctions')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(3)
                                   .transform.missing.fill()
                                   )

                      .with_column(Column('topic2_sanctions_t15', from_table='topic_cm', from_column='topic_sanctions')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(15)
                                   .transform.missing.fill()
                                   )

                      .with_column(Column('topic2_sanctions_t3_stock', from_table='topic_cm',
                                          from_column='topic_sanctions')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(3)
                                   .transform.missing.fill()
                                   .transform.temporal.moving_average(12)
                                   .transform.missing.fill()
                                   )

                      .with_column(Column('topic2_sanctions_t3_stock', from_table='topic_cm',
                                          from_column='topic_sanctions')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(3)
                                   .transform.missing.fill()
                                   .transform.temporal.moving_average(12)
                                   .transform.missing.fill()
                                   .transform.spatial.countrylag(1, 1, 0, 0)
                                   .transform.missing.replace_na()
                                   )

                      # Topic 3, civilian life: original, tlag1, 12 month moving average on tlag1
                      .with_column(Column('topic3_life_t3', from_table='topic_cm', from_column='topic_life')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(3)
                                   .transform.missing.fill()
                                   )

                      .with_column(Column('topic3_life_t15', from_table='topic_cm', from_column='topic_life')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(15)
                                   .transform.missing.fill()
                                   )
                      .with_column(Column('topic3_life_t3_stock', from_table='topic_cm', from_column='topic_life')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(3)
                                   .transform.missing.fill()
                                   .transform.temporal.moving_average(12)
                                   .transform.missing.fill()
                                   )

                      .with_column(Column('splag_topic3_life_t3_stock', from_table='topic_cm', from_column='topic_life')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(3)
                                   .transform.missing.fill()
                                   .transform.temporal.moving_average(15)
                                   .transform.missing.fill()
                                   .transform.spatial.countrylag(1, 1, 0, 0)
                                   .transform.missing.replace_na()
                                   )

                      # Topic 4, energy and industry: original, tlag1, 12 month moving average on tlag1
                      .with_column(Column('topic4_energy_t3', from_table='topic_cm', from_column='topic_energy')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(3)
                                   .transform.missing.fill()
                                   )

                      .with_column(Column('topic4_energy_t15', from_table='topic_cm', from_column='topic_energy')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(15)
                                   .transform.missing.fill()
                                   )

                      .with_column(Column('topic4_energy_t3_stock', from_table='topic_cm', from_column='topic_energy')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(3)
                                   .transform.missing.fill()
                                   .transform.temporal.moving_average(12)
                                   .transform.missing.fill()
                                   )

                      .with_column(Column('splag_topic4_energy_t3_stock', from_table='topic_cm',
                                          from_column='topic_energy')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(3)
                                   .transform.missing.fill()
                                   .transform.temporal.moving_average(12)
                                   .transform.missing.fill()
                                   .transform.spatial.countrylag(1, 1, 0, 0)
                                   .transform.missing.replace_na()
                                   )

                      # Topic 5, media and reporting: original, tlag1, 12 month moving average on tlag1
                      .with_column(Column('topic5_media_t3', from_table='topic_cm', from_column='topic_media')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(3)
                                   .transform.missing.fill()
                                   )

                      .with_column(Column('topic5_media_t15', from_table='topic_cm', from_column='topic_media')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(15)
                                   .transform.missing.fill()
                                   )

                      .with_column(Column('topic5_media_t3_stock', from_table='topic_cm', from_column='topic_media')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(3)
                                   .transform.missing.fill()
                                   .transform.temporal.moving_average(12)
                                   .transform.missing.fill()
                                   )

                      .with_column(Column('splag_topic5_media_t3_stock', from_table='topic_cm',
                                          from_column='topic_media')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(3)
                                   .transform.missing.fill()
                                   .transform.temporal.moving_average(12)
                                   .transform.missing.fill()
                                   .transform.spatial.countrylag(1, 1, 0, 0)
                                   .transform.missing.replace_na()
                                   )

                      # Topic 6, economics: original, tlag1, 12 month moving average on tlag1
                      .with_column(Column('topic6_economics_t3', from_table='topic_cm', from_column='topic_economics')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(3)
                                   .transform.missing.fill()
                                   )

                      .with_column(Column('topic6_economics_t15', from_table='topic_cm', from_column='topic_economics')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(15)
                                   .transform.missing.fill()
                                   )

                      .with_column(Column('topic6_economics_t3_stock', from_table='topic_cm',
                                          from_column='topic_economics')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(3)
                                   .transform.missing.fill()
                                   .transform.temporal.moving_average(15)
                                   .transform.missing.fill()
                                   )

                      .with_column(Column('splag_topic6_economics_t3_stock', from_table='topic_cm',
                                          from_column='topic_economics')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(3)
                                   .transform.missing.fill()
                                   .transform.temporal.moving_average(12)
                                   .transform.missing.fill()
                                   .transform.spatial.countrylag(1, 1, 0, 0)
                                   .transform.missing.replace_na()
                                   )

                      # Topic 7, health and emergencies: original, tlag1, 12 month moving average on tlag1
                      .with_column(Column('topic7_health_t3', from_table='topic_cm', from_column='topic_health')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(3)
                                   .transform.missing.fill()
                                   )

                      .with_column(Column('topic7_health_t15', from_table='topic_cm', from_column='topic_health')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(15)
                                   .transform.missing.fill()
                                   )

                      .with_column(Column('topic7_health_t3_stock', from_table='topic_cm', from_column='topic_health')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(3)
                                   .transform.missing.fill()
                                   .transform.temporal.moving_average(12)
                                   .transform.missing.fill()
                                   )

                      .with_column(Column('splag_topic7_health_t3_stock', from_table='topic_cm',
                                          from_column='topic_health')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(3)
                                   .transform.missing.fill()
                                   .transform.temporal.moving_average(12)
                                   .transform.missing.fill()
                                   .transform.spatial.countrylag(1, 1, 0, 0)
                                   .transform.missing.replace_na()
                                   )

                      # Topic 8, chinese politics: original, tlag1, 12 month moving average on tlag1
                      .with_column(Column('topic8_china_t3', from_table='topic_cm', from_column='topic_china')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(3)
                                   .transform.missing.fill()
                                   )

                      .with_column(Column('topic8_china_t15', from_table='topic_cm', from_column='topic_china')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(15)
                                   .transform.missing.fill()
                                   )

                      .with_column(Column('topic8_china_t3_stock', from_table='topic_cm', from_column='topic_china')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(3)
                                   .transform.missing.fill()
                                   .transform.temporal.moving_average(12)
                                   .transform.missing.fill()
                                   )

                      .with_column(Column('splag_topic8_china_t3_stock', from_table='topic_cm',
                                          from_column='topic_china')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(3)
                                   .transform.missing.fill()
                                   .transform.temporal.moving_average(12)
                                   .transform.missing.fill()
                                   .transform.spatial.countrylag(1, 1, 0, 0)
                                   .transform.missing.replace_na()
                                   )

                      # Topic 9, foreign policy: original, tlag1, 12 month moving average on tlag1
                      .with_column(Column('topic9_foreign_t3', from_table='topic_cm', from_column='topic_foreign')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(3)
                                   .transform.missing.fill()
                                   )

                      .with_column(Column('topic9_foreign_t15', from_table='topic_cm', from_column='topic_foreign')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(15)
                                   .transform.missing.fill()
                                   )

                      .with_column(Column('topic9_foreign_t3_stock', from_table='topic_cm',
                                          from_column='topic_foreign')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(3)
                                   .transform.missing.fill()
                                   .transform.temporal.moving_average(12)
                                   .transform.missing.fill()
                                   )

                      .with_column(Column('splag_topic9_foreign_t3_stock', from_table='topic_cm',
                                          from_column='topic_foreign')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(3)
                                   .transform.missing.fill()
                                   .transform.temporal.moving_average(12)
                                   .transform.missing.fill()
                                   .transform.spatial.countrylag(1, 1, 0, 0)
                                   .transform.missing.replace_na()
                                   )

                      # Topic 10, armed conflict: original, tlag1, 12 month moving average on tlag1
                      .with_column(Column('topic10_conflict_t3', from_table='topic_cm', from_column='topic_conflict')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(3)
                                   .transform.missing.fill()
                                   )

                      .with_column(Column('topic10_conflict_t15', from_table='topic_cm', from_column='topic_conflict')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(15)
                                   .transform.missing.fill()
                                   )

                      .with_column(Column('topic10_conflict_t3_stock', from_table='topic_cm',
                                          from_column='topic_conflict')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(3)
                                   .transform.missing.fill()
                                   .transform.temporal.moving_average(12)
                                   .transform.missing.fill()
                                   )

                      .with_column(Column('splag_topic10_conflict_t3_stock', from_table='topic_cm',
                                          from_column='topic_conflict')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(3)
                                   .transform.missing.fill()
                                   .transform.temporal.moving_average(12)
                                   .transform.missing.fill()
                                   .transform.spatial.countrylag(1, 1, 0, 0)
                                   .transform.missing.replace_na()
                                   )

                      # Topic 11, diplomacy: original, tlag1, 12 month moving average on tlag1
                      .with_column(Column('topic11_diplomacy_t3', from_table='topic_cm', from_column='topic_diplomacy')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(3)
                                   .transform.missing.fill()
                                   )

                      .with_column(Column('topic11_diplomacy_t15', from_table='topic_cm', from_column='topic_diplomacy')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(15)
                                   .transform.missing.fill()
                                   )

                      .with_column(Column('topic11_diplomacy_t3_stock', from_table='topic_cm',
                                          from_column='topic_diplomacy')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(3)
                                   .transform.missing.fill()
                                   .transform.temporal.moving_average(12)
                                   .transform.missing.fill()
                                   )

                      .with_column(Column('splag_topic11_diplomacy_t3_stock', from_table='topic_cm',
                                          from_column='topic_diplomacy')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(3)
                                   .transform.missing.fill()
                                   .transform.temporal.moving_average(12)
                                   .transform.missing.fill()
                                   .transform.spatial.countrylag(1, 1, 0, 0)
                                   .transform.missing.replace_na()
                                   )

                      # Topic 12, power and negotiation: original, tlag1, 12 month moving average on tlag1
                      .with_column(Column('topic12_power_t3', from_table='topic_cm', from_column='topic_power')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(3)
                                   .transform.missing.fill()
                                   )

                      .with_column(Column('topic12_power_t15', from_table='topic_cm', from_column='topic_power')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(15)
                                   .transform.missing.fill()
                                   )

                      .with_column(Column('topic12_power_t3_stock', from_table='topic_cm', from_column='topic_power')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(3)
                                   .transform.missing.fill()
                                   .transform.temporal.moving_average(12)
                                   .transform.missing.fill()
                                   )

                      .with_column(Column('splag_topic12_power_t3_stock', from_table='topic_cm',
                                          from_column='topic_power')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(3)
                                   .transform.missing.fill()
                                   .transform.temporal.moving_average(12)
                                   .transform.missing.fill()
                                   .transform.spatial.countrylag(1, 1, 0, 0)
                                   .transform.missing.replace_na()
                                   )

                      # Topic 13, sports: original, tlag1, 12 month moving average on tlag1
                      .with_column(Column('topic13_sports_t3', from_table='topic_cm', from_column='topic_sports')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(3)
                                   .transform.missing.fill()
                                   )

                      .with_column(Column('topic13_sports_t15', from_table='topic_cm', from_column='topic_sports')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(15)
                                   .transform.missing.fill()
                                   )

                      .with_column(Column('topic13_sports_t3_stock', from_table='topic_cm', from_column='topic_sports')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(3)
                                   .transform.missing.fill()
                                   .transform.temporal.moving_average(12)
                                   .transform.missing.fill()
                                   )

                      .with_column(Column('splag_topic13_sports_t3_stock', from_table='topic_cm',
                                          from_column='topic_sports')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(3)
                                   .transform.missing.fill()
                                   .transform.temporal.moving_average(12)
                                   .transform.missing.fill()
                                   .transform.spatial.countrylag(1, 1, 0, 0)
                                   .transform.missing.replace_na()
                                   )

                      # Topic 14, judiciary and abuses: original, tlag1, 12 month moving average on tlag1
                      .with_column(Column('topic14_judiciary_t3', from_table='topic_cm', from_column='topic_judiciary')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(3)
                                   .transform.missing.fill()
                                   )

                      .with_column(Column('topic14_judiciary_t15', from_table='topic_cm', from_column='topic_judiciary')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(15)
                                   .transform.missing.fill()
                                   )

                      .with_column(Column('topic14_judiciary_t3_stock', from_table='topic_cm',
                                          from_column='topic_judiciary')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(3)
                                   .transform.missing.fill()
                                   .transform.temporal.moving_average(12)
                                   .transform.missing.fill()
                                   )

                      .with_column(Column('splag_topic14_judiciary_t3_stock', from_table='topic_cm',
                                          from_column='topic_judiciary')
                                   .transform.missing.fill()
                                   .transform.missing.replace_na()
                                   .transform.temporal.tlag(3)
                                   .transform.missing.fill()
                                   .transform.temporal.moving_average(12)
                                   .transform.missing.fill()
                                   .transform.spatial.countrylag(1, 1, 0, 0)
                                   .transform.missing.replace_na()
                                   )
                      .with_theme("fatalities")
                      .describe("""Predicting ln(fatalities), cm level
    
                                Stub queryset with Mueller & Rauh topic model features
    
                                """)
                      )

    data = qs_topics_stub.publish().fetch()

    print(f"fat_dev_mc_media_topics; "
          f"A dataset with {len(data.columns)} columns, with "
          f"data between t {min(data.index.get_level_values(0))} "
          f"and {max(data.index.get_level_values(0))}. "
          f"({len(np.unique(data.index.get_level_values(1)))} units)"
          )
    
    ###Google trends and internet use queryset###

    qs_google_internet = (Queryset("fatalities_mc_google_internet", "country_month")

                   # population
                   .with_column(Column("wdi_sp_pop_totl", from_table = "wdi_cy", from_column = "wdi_sp_pop_totl")
                                .transform.missing.fill()
                                .transform.temporal.tlag(12)
                                .transform.missing.fill())  

                   #google trends
                   .with_column(Column("google_index_conflict_t2", from_table="gogle_trend_cm", from_column="google_index_conflict")
                                .transform.missing.fill()
                                .transform.missing.replace_na()
                                .transform.temporal.tlag(2)
                                .transform.missing.fill()
                                )
                    
                   .with_column(Column("google_index_conflict_t14", from_table="gogle_trend_cm", from_column="google_index_conflict")
                                .transform.missing.fill()
                                .transform.missing.replace_na()
                                .transform.temporal.tlag(14)
                                .transform.missing.fill()
                                )
                    .with_column(Column("google_index_conflict_t2_ra", from_table="gogle_trend_cm", from_column="google_index_conflict")
                                .transform.missing.fill()
                                .transform.missing.replace_na()
                                .transform.temporal.tlag(2)
                                .transform.missing.fill()
                                .transform.temporal.moving_average(12)
                                .transform.missing.fill()
                                )
                    .with_column(Column("google_index_conflict_t2_ra_splag", from_table="gogle_trend_cm", from_column="google_index_conflict")
                                .transform.missing.fill()
                                .transform.missing.replace_na()
                                .transform.temporal.tlag(3)
                                .transform.missing.fill()
                                .transform.temporal.moving_average(12)
                                .transform.missing.fill()
                                .transform.spatial.countrylag(1, 1, 0, 0)
                                .transform.missing.replace_na()
                                )
                    .with_column(Column("google_index_war_t2", from_table="gogle_trend_cm", from_column="google_index_war")
                                .transform.missing.fill()
                                .transform.missing.replace_na()
                                .transform.temporal.tlag(2)
                                .transform.missing.fill()
                                )
                    
                   .with_column(Column("google_index_war_t14", from_table="gogle_trend_cm", from_column="google_index_war")
                                .transform.missing.fill()
                                .transform.missing.replace_na()
                                .transform.temporal.tlag(14)
                                .transform.missing.fill()
                                )
                    .with_column(Column("google_index_war_t2_ra", from_table="gogle_trend_cm", from_column="google_index_war")
                                .transform.missing.fill()
                                .transform.missing.replace_na()
                                .transform.temporal.tlag(2)
                                .transform.missing.fill()
                                .transform.temporal.moving_average(12)
                                .transform.missing.fill()
                                )
                    .with_column(Column("google_index_war_t2_ra_splag", from_table="gogle_trend_cm", from_column="google_index_war")
                                .transform.missing.fill()
                                .transform.missing.replace_na()
                                .transform.temporal.tlag(3)
                                .transform.missing.fill()
                                .transform.temporal.moving_average(12)
                                .transform.missing.fill()
                                .transform.spatial.countrylag(1, 1, 0, 0)
                                .transform.missing.replace_na()
                                )
                    #internet usage
                    .with_column(Column("internet_t24", from_table="internet_usage_our_world_data_cy", from_column="owd_internet_pop_percent")
                                .transform.missing.fill()
                                .transform.missing.replace_na()
                                .transform.temporal.tlag(24)
                                .transform.missing.fill()
                                )
                    
                   .with_column(Column("internet_t36", from_table="internet_usage_our_world_data_cy", from_column="owd_internet_pop_percent")
                                .transform.missing.fill()
                                .transform.missing.replace_na()
                                .transform.temporal.tlag(36)
                                .transform.missing.fill()
                                )
                    .with_column(Column("internet_t24_ra", from_table="internet_usage_our_world_data_cy", from_column="owd_internet_pop_percent")
                                .transform.missing.fill()
                                .transform.missing.replace_na()
                                .transform.temporal.tlag(24)
                                .transform.missing.fill()
                                .transform.temporal.moving_average(12)
                                .transform.missing.fill()
                                )
                    .with_column(Column("internet_t24_ra_splag", from_table="internet_usage_our_world_data_cy", from_column="owd_internet_pop_percent")
                                .transform.missing.fill()
                                .transform.missing.replace_na()
                                .transform.temporal.tlag(24)
                                .transform.missing.fill()
                                .transform.temporal.moving_average(12)
                                .transform.missing.fill()
                                .transform.spatial.countrylag(1, 1, 0, 0)
                                .transform.missing.replace_na()
                                )

                   .with_theme("fatalities")
                   .describe("""Fatalities conflict history, cm level
    
                   Predicting ln(fatalities) using conflict predictors, ultrashort
    
                             """)
                   )

    data = qs_google_internet.publish().fetch()

    print(f"fat_dev_mc_media_google_internet; "
          f"A dataset with {len(data.columns)} columns, with "
          f"data between t {min(data.index.get_level_values(0))} "
          f"and {max(data.index.get_level_values(0))}. "
          f"({len(np.unique(data.index.get_level_values(1)))} units)"
          )